<a href="https://colab.research.google.com/github/edwin-91003/Electrical-load-forecasting-LSTm/blob/main/Internship_Load_Forecasting_2023_2024_Batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ***# Implement ANN with keras - Regression Problem***

---



In [2]:
import numpy as np
import matplotlib.pyplot as plt
import keras  #Keras is the deep learning library that helps you to code Deep Neural Networks with fewer lines of code
#from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop,Adadelta,SGD,Adagrad,Adam
#import pylab as plt
#import seaborn as sns #For data visualization
import pandas as pd # For Data manipulation

In [3]:
load_data=pd.read_csv("/content/drive/MyDrive/Original Active Power Load DataSet.csv")
load_data.head()
min_load=load_data.min()
print(min_load)
max_load=load_data.max()
print(max_load)

T-1         458.020052
T-2         455.976231
T-3         455.976231
T-4         455.976231
T-24        455.976231
T-48        431.654774
T-72        431.654774
T-96        412.034103
DAY           0.000000
SEASON        0.000000
TEMP         50.000000
HUMIDITY     14.000000
T           458.020052
dtype: float64
T-1         6306.206465
T-2         6306.206465
T-3         6306.206465
T-4         6306.206465
T-24        6306.206465
T-48        6306.206465
T-72        6306.206465
T-96        6306.206465
DAY            1.000000
SEASON         2.000000
TEMP         108.000000
HUMIDITY     102.000000
T           6306.206465
dtype: float64


In [4]:
load_data.head()

,T-1,T-2,T-3,T-4,T-24,T-48,T-72,T-96,DAY,SEASON,TEMP,HUMIDITY,T
0,2083.646729,455.976231,515.451392,654.226767,2260.056104,2236.756556,1828.915749,1967.388015,0,1,66,90,2062.165835
1,2062.165835,2083.646729,455.976231,515.451392,2260.056104,2236.756556,1828.915749,1967.388015,0,1,66,90,2062.165835
2,2062.165835,2062.165835,2083.646729,455.976231,2260.056104,2354.480586,1828.915749,1967.388015,0,1,66,90,2062.165835
3,2062.165835,2062.165835,2062.165835,2083.646729,2555.415800,2511.445958,1892.265507,2442.607331,0,1,80,40,2294.620910
4,2294.620910,2062.165835,2062.165835,2062.165835,2845.399210,2805.756031,2532.344883,2756.205522,0,1,80,40,2710.645657


In [5]:
print(load_data.shape) # details about number of samples and features
load_data.describe()

(8664, 13)


,T-1,T-2,T-3,T-4,T-24,T-48,T-72,T-96,DAY,SEASON,TEMP,HUMIDITY,T
count,8664.000000,8664.000000,8664.000000,8664.000000,8664.000000,8664.000000,8664.000000,8664.000000,8664.000000,8664.000000,8664.000000,8664.000000,8664.000000
mean,2121.265802,2121.213596,2121.148598,2121.088698,2121.386849,2121.235808,2120.310528,2119.569988,0.141274,0.997230,81.288666,67.718837,2121.122241
std,1294.262887,1294.320941,1294.387443,1294.443323,1294.501059,1295.442808,1295.382983,1295.149266,0.348324,0.823857,8.843980,20.659699,1294.336041
min,458.020052,455.976231,455.976231,455.976231,455.976231,431.654774,431.654774,412.034103,0.000000,0.000000,50.000000,14.000000,458.020052
25%,1062.786376,1062.786376,1062.753900,1062.656472,1062.656472,1061.934207,1061.480409,1061.480409,0.000000,0.000000,77.000000,52.000000,1062.753900
50%,1673.193123,1673.193123,1673.193123,1673.193123,1673.435610,1673.193123,1673.161080,1673.161080,0.000000,1.000000,81.000000,72.000000,1673.161080
75%,3001.301103,3001.301103,3001.301103,3001.301103,3004.839684,3006.340505,3006.340505,3007.149806,0.000000,2.000000,86.000000,87.000000,3001.301103
max,6306.206465,6306.206465,6306.206465,6306.206465,6306.206465,6306.206465,6306.206465,6306.206465,1.000000,2.000000,108.000000,102.000000,6306.206465


In [6]:
load_data.isnull().any()

T-1         False
T-2         False
T-3         False
T-4         False
T-24        False
T-48        False
T-72        False
T-96        False
DAY         False
SEASON      False
TEMP        False
HUMIDITY    False
T           False
dtype: bool

In [7]:
load_data = load_data.fillna(method='ffill')

# **Outlier Treatment**

# Normalization

In [8]:
from sklearn.preprocessing import MinMaxScaler
ms=MinMaxScaler()
load_data=ms.fit_transform(load_data)
print(load_data)

[[0.27797108 0.         0.01016629 ... 0.27586207 0.86363636 0.27429799]
 [0.27429799 0.27822332 0.         ... 0.27586207 0.86363636 0.27429799]
 [0.27429799 0.27455152 0.27822332 ... 0.27586207 0.86363636 0.27429799]
 ...
 [0.12229168 0.13086742 0.13413007 ... 0.25862069 0.90909091 0.10611346]
 [0.10611346 0.12259832 0.13086742 ... 0.25862069 0.90909091 0.07699265]
 [0.07699265 0.10642574 0.12259832 ... 0.25862069 0.90909091 0.06528806]]


In [9]:
dataset=pd.DataFrame(data=load_data[0:,0:])
print(dataset[1].values)

[0.         0.27822332 0.27455152 ... 0.13086742 0.12259832 0.10642574]


In [10]:
X=dataset.iloc[:,0:12].values
Y=dataset.iloc[:,12:].values
print(X)
print(Y)

[[0.27797108 0.         0.01016629 ... 0.5        0.27586207 0.86363636]
 [0.27429799 0.27822332 0.         ... 0.5        0.27586207 0.86363636]
 [0.27429799 0.27455152 0.27822332 ... 0.5        0.27586207 0.86363636]
 ...
 [0.12229168 0.13086742 0.13413007 ... 0.5        0.25862069 0.90909091]
 [0.10611346 0.12259832 0.13086742 ... 0.5        0.25862069 0.90909091]
 [0.07699265 0.10642574 0.12259832 ... 0.5        0.25862069 0.90909091]]
[[0.27429799]
 [0.27429799]
 [0.27429799]
 ...
 [0.10611346]
 [0.07699265]
 [0.06528806]]


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print(y_test[0:5])
print(X_test.shape)
print(X_train.shape)

[[0.16772695]
 [0.23749782]
 [0.88243576]
 [0.12113366]
 [0.25144501]]
(1733, 12)
(6931, 12)


In [12]:
#First_Layer_Size = 32 # Number of neurons in first layer
model=Sequential()
model.add(Dense(24,activation='sigmoid', input_shape=(12,)))
model.add(Dense(24,activation='sigmoid'))
model.add(Dense(24,activation='sigmoid'))
model.add(Dense(24,activation='sigmoid'))
model.add(Dense(24,activation='sigmoid'))

model.add(Dense(1,activation='sigmoid')) #output Layer
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                312       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 24)                600       
                                                                 
 dense_3 (Dense)             (None, 24)                600       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 1)                 25        
                                                                 
Total params: 2737 (10.69 KB)
Trainable params: 2737 (10

In [13]:
model.compile(loss='MSE',
              optimizer=Adam(),
              metrics=['MSE'])

In [14]:
# Write the Training input and output variables, size of the batch, number of epochs
history = model.fit(X_train,y_train,
                    batch_size=8,
                    epochs=200,verbose=1)

Epoch 1/200
867/867 [==============================] - 3s 2ms/step - loss: 0.0505 - MSE: 0.0505
Epoch 2/200
867/867 [==============================] - 2s 2ms/step - loss: 0.0092 - MSE: 0.0092
Epoch 3/200
867/867 [==============================] - 2s 3ms/step - loss: 0.0077 - MSE: 0.0077
Epoch 4/200
867/867 [==============================] - 3s 3ms/step - loss: 0.0066 - MSE: 0.0066
Epoch 5/200
867/867 [==============================] - 3s 4ms/step - loss: 0.0060 - MSE: 0.0060
Epoch 6/200
867/867 [==============================] - 2s 2ms/step - loss: 0.0056 - MSE: 0.0056
Epoch 7/200
867/867 [==============================] - 2s 2ms/step - loss: 0.0053 - MSE: 0.0053
Epoch 8/200
867/867 [==============================] - 2s 2ms/step - loss: 0.0051 - MSE: 0.0051
Epoch 9/200
867/867 [==============================] - 2s 2ms/step - loss: 0.0050 - MSE: 0.0050
Epoch 10/200
867/867 [==============================] - 5s 6ms/step - loss: 0.0049 - MSE: 0.0049
Epoch 11/200
867/867 [=================

In [15]:
 # Write the testing input and output variables
score = model.evaluate(X_test, y_test, verbose=2)
print('Test loss:', score[0])

55/55 - 0s - loss: 0.0035 - MSE: 0.0035 - 230ms/epoch - 4ms/step
Test loss: 0.0035121855325996876


In [16]:
# Write the index of the test sample to test
print(X_test[0])
prediction = model.predict(X_test[0].reshape(1,12))
print(prediction[0]*(max_load-min_load)+min_load)
print(y_test[0]*(max_load-min_load)+min_load)

[0.15804815 0.1524352  0.1524352  0.1524352  0.15068752 0.14521856
 0.04451607 0.15463347 1.         1.         0.84482759 0.38636364]
1/1 [==============================] - 0s 104ms/step
T-1         1317.094660
T-2         1315.351069
T-3         1315.351069
T-4         1315.351069
T-24        1315.351069
T-48        1294.602334
T-72        1294.602334
T-96        1277.863859
DAY            0.146896
SEASON         0.293792
TEMP          58.519962
HUMIDITY      26.926839
T           1317.094660
dtype: float64
T-1         1438.918529
T-2         1437.217513
T-3         1437.217513
T-4         1437.217513
T-24        1437.217513
T-48        1416.975419
T-72        1416.975419
T-96        1400.645663
DAY            0.167727
SEASON         0.335454
TEMP          59.728163
HUMIDITY      28.759972
T           1438.918529
dtype: float64


In [17]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))


In [18]:
# load model
import pickle
pickled_model = pickle.load(open('model.pkl', 'rb'))

In [22]:
import pickle
model = pickle.load(open('model.pkl','rb'))
inputvector=np.zeros((1,6))
for j in range(6):
  A=input("enter value : ")
  inputvector[0,j]=(float(A)-min_load)/(max_load-min_load)
prediction = model.predict(inputvector[0].reshape(1,6))
print("Predcited Load=",prediction[0]*(max_load-min_load)+min_load)


enter value : 1


ValueError: ignored

In [ ]:
 min_load

T-1         458.020052
T-2         455.976231
T-3         455.976231
T-4         455.976231
T-24        455.976231
T-48        431.654774
T-72        431.654774
T-96        412.034103
DAY           0.000000
SEASON        0.000000
TEMP         50.000000
HUMIDITY     14.000000
T           458.020052
dtype: float64

In [ ]:
max_load

T-1         6306.206465
T-2         6306.206465
T-3         6306.206465
T-4         6306.206465
T-24        6306.206465
T-48        6306.206465
T-72        6306.206465
T-96        6306.206465
DAY            1.000000
SEASON         2.000000
TEMP         108.000000
HUMIDITY     102.000000
T           6306.206465
dtype: float64

In [24]:
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle

# Create flask app
flask_app = Flask(__name__)
model = pickle.load(open("model.pkl", "rb"))

@flask_app.route("/")
def Home():
    return render_template("login.html")

@flask_app.route("/predict", methods = ["POST"])
def predict():
    float_features = [float(x) for x in request.form.values()]
    features = [np.array(float_features)]
    prediction = model.predict(features)
    return render_template("login.html", prediction_text = "The predicted load is {}".format(prediction))

if __name__ == "__main__":
    flask_app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
